# Sample Output from the Model and Publish on Hugging Face

## Setup & Imports

In [ ]:
import os
from dotenv import load_dotenv
from huggingface_hub import login
from transformers import pipeline
from peft import PeftConfig, PeftModel
from twitter_emoji_reaction_lora.model import build_inference_peft_model

In [ ]:
load_dotenv()

login(token=os.getenv("HUGGINGFACE_TOKEN"))

In [ ]:
model = build_base_model()
model_id = "roberta-base-with-tweet-eval-emoji" #path to model in local
build_inference_peft_model(model, model_id)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [142]:
id2label = {
    0: "❤", 1: "😍", 2: "😂", 3: "💕", 4: "🔥",
    5: "😊", 6: "😎", 7: "✨", 8: "💙", 9: "😘",
    10: "📷", 11: "🇺🇸", 12: "☀", 13: "💜", 14: "😉",
    15: "💯", 16: "😁", 17: "🎄", 18: "📸", 19: "😜"
}

In [ ]:
pipe = pipeline(
    task="text-classification",
    model=model,
    tokenizer=tok,
    return_all_scores=True,          # return all 20 logits
    function_to_apply="softmax"      # turn logits -> probabilities
)

def emojify(text, k=3):
    probs = pipe(text)[0]
    top = sorted(probs, key=lambda x: x["score"], reverse=True)[:k]
    return " ".join(id2label[int(d["label"].split("_")[-1])] for d in top)


print(emojify("Sunny day"))

Device set to use cuda:0


☀ 💙 😎


## Push to Huggingface

Uncomment to use code

In [1]:
# repo_id = "your_username/roberta-base-tweet-emoji-lora"

# trainer.create_model_card(
#     model_name=repo_id.split("/")[-1],
#     language="en",
#     license="apache-2.0",
#     tags=["twitter", "emoji", "lora", "roberta-base"],
# )

# trainer.push_to_hub(
#     repo_id,
#     finetuned_from="roberta-base",
#     tasks="text-classification",
#     dataset="tweet_eval/emoji",
# )